# 第三章 索引


In [17]:
import numpy as np
import pandas as pd
%cd drive/MyDrive/
!ls

[Errno 2] No such file or directory: 'drive/MyDrive/'
/content/drive/MyDrive
'Colab Notebooks'   data


## 一、索引器

### 1.表的列索引

一般通过 [ ] 实现，返回值是Series

In [ ]:
df = pd.read_csv('data/learn_pandas.csv',usecols = ['School', 'Grade', 'Name', 'Gender','Weight', 'Transfer'])
df['Name'].head()

0      Gaopeng Yang
1    Changqiang You
2           Mei Sun
3      Xiaojuan Sun
4       Gaojuan You
Name: Name, dtype: object

取多个列，用[列名组成的列表]，返回值是个DataFrame

In [18]:
df[['Gender', 'Name']].head()

,Gender,Name
0,Female,Gaopeng Yang
1,Male,Changqiang You
2,Male,Mei Sun
3,Female,Xiaojuan Sun
4,Male,Gaojuan You


取单列且列名中不包含空格，则可以用 .列名 取出，等价于[列名]

In [19]:
df.Name.head()
#等价于 df['Name'].head()

0      Gaopeng Yang
1    Changqiang You
2           Mei Sun
3      Xiaojuan Sun
4       Gaojuan You
Name: Name, dtype: object

### 2.序列的行索引

（1）以字符串为索引的 Series

取出单个索引的对应元素，用 [item] ，若 Series 只有单个值对应，则返回这个标量值，如果有多个值对应，则返回一个 Series。

对应的，取多个索引的对应元素就用[items的列表]

取出某两个索引之间的元素，且这两个索引是唯一出现，则可以使用切片。注意：这里的切片会包含两个端点，如果前后端点的值存在重复，即非唯一值，那么需要经过排序才能使用切片。

In [22]:
s = pd.Series(['a', 'b', 'c', 'd', 'e', 'f'],index=[1, 3, 1, 2, 5, 4])
try:
   s['a': 'b']
except Exception as e:
   Err_Msg = e
Err_Msg
s.sort_index()

1    a
1    c
2    d
3    b
4    f
5    e
dtype: object

（2）以整数为索引的Series

读入函数时，如果不特别指定，则会生成从0开始的整数索引作为默认索引。任意一组符合长度要求的整数都可以作为索引。

和字符串一样，如果使用 [int] 或 [int_list] ，则可以取出对应索引元素的值

In [23]:
s = pd.Series(['a', 'b', 'c', 'd', 'e', 'f'],
   index=[1, 3, 1, 2, 5, 4])
s[1]
s[[2,3]]

2    d
3    b
dtype: object

如果使用整数切片，则会取出对应索引位置的值

**注意这里的整数切片同 Python 中的切片一样不包含右端点**

In [24]:
s[1:-1:2]

3    b
2    d
dtype: object

注意！！

不要把纯浮点以及任何混合类型（字符串、整数、浮点类型等的混合）作为索引，否则可能会在具体的操作时报错或者返回非预期的结果，实际也无该操作需求。

### 3.loc 索引器
对于表而言，有两种索引器，一种是基于 元素 的 loc 索引器，另一种是基于 位置 的 iloc 索引器

loc 索引器的一般形式是 loc[*, *] ，其中第一个 * 代表行的选择，第二个 * 代表列的选择，

如果省略第二个位置写作 loc[*] ，这个 * 是指行的筛选。

其中， * 的位置一共有五类合法对象，分别是：单个元素、元素列表、元素切片、布尔列表以及函数

（1）* 为单个元素

此时，直接取出相应的行或列，如果该元素在索引中重复则结果为 DataFrame，否则为 Series

也可同时选择行和列

In [26]:
df_demo = df.set_index('Name')
df_demo.loc['Qiang Sun'] # 多个人叫此名字
df_demo.loc['Quan Zhao'] # 名字唯一
df_demo.loc['Qiang Sun', 'School'] # 返回Series
df_demo.loc['Quan Zhao', 'School'] # 返回单个元素

'Shanghai Jiao Tong University'

（2）*为元素列表

此时，取出列表中所有元素值对应的行或列

In [27]:
df_demo.loc[['Qiang Sun','Quan Zhao'], ['School','Gender']]

,School,Gender
Name,,
Qiang Sun,Tsinghua University,Female
Qiang Sun,Tsinghua University,Female
Qiang Sun,Shanghai Jiao Tong University,Female
Quan Zhao,Shanghai Jiao Tong University,Female


（3）*为切片

如果是唯一值的起点和终点字符，那么就可以使用切片，并且包含两个端点，如果不唯一则报错

In [28]:
 df_demo.loc['Gaojuan You':'Gaoqiang Qian', 'School':'Gender']

,School,Grade,Gender
Name,,,
Gaojuan You,Fudan University,Sophomore,Male
Xiaoli Qian,Tsinghua University,Freshman,Female
Qiang Chu,Shanghai Jiao Tong University,Freshman,Female
Gaoqiang Qian,Tsinghua University,Junior,Female


注意！！如果 DataFrame 使用整数索引，其使用整数切片的时候和上面字符串索引的要求一致，都是 元素切片，包含端点且起点、终点不允许有重复值。

In [29]:
df_loc_slice_demo = df_demo.copy()
df_loc_slice_demo.index = range(df_demo.shape[0],0,-1)
df_loc_slice_demo.loc[5:3]
df_loc_slice_demo.loc[3:5] # 没有返回，说明不是整数位置切片

,School,Grade,Gender,Weight,Transfer


（4）*为布尔列表

传入 loc 的布尔列表与 DataFrame 长度相同，且列表为 True 的位置所对应的行会被选中， False 则会被剔除。

In [30]:
df_demo.loc[df_demo.Weight>70].head()
# 也可以通过 isin 方法返回的布尔列表等价写出
# df_demo.loc[df_demo.Grade.isin(['Freshman', 'Senior'])].head()

,School,Grade,Gender,Weight,Transfer
Name,,,,,
Mei Sun,Shanghai Jiao Tong University,Senior,Male,89.0,N
Gaojuan You,Fudan University,Sophomore,Male,74.0,N
Xiaopeng Zhou,Shanghai Jiao Tong University,Freshman,Male,74.0,N
Xiaofeng Sun,Tsinghua University,Senior,Male,71.0,N
Qiang Zheng,Shanghai Jiao Tong University,Senior,Male,87.0,N


对于复合条件而言，可以用 |（或）, &（且）, ~（取反） 的组合来实现

In [31]:
# 选出复旦大学中体重超过70kg的大四学生，或者北大男生中体重超过80kg的非大四的学生
condition_1_1 = df_demo.School == 'Fudan University'
condition_1_2 = df_demo.Grade == 'Senior'
condition_1_3 = df_demo.Weight > 70
condition_1 = condition_1_1 & condition_1_2 & condition_1_3
condition_2_1 = df_demo.School == 'Peking University'
condition_2_2 = df_demo.Grade == 'Senior'
condition_2_3 = df_demo.Weight > 80
condition_2 = condition_2_1 & (~condition_2_2) & condition_2_3
df_demo.loc[condition_1 | condition_2]

,School,Grade,Gender,Weight,Transfer
Name,,,,,
Qiang Han,Peking University,Freshman,Male,87.0,N
Chengpeng Zhou,Fudan University,Senior,Male,81.0,N
Changpeng Zhao,Peking University,Freshman,Male,83.0,N
Chengpeng Qian,Fudan University,Senior,Male,73.0,Y


（5）*为函数

这里的函数，必须以前面的四种合法形式之一为返回值，并且函数的输入值为 DataFrame 本身

可以把逻辑写入一个函数中再返回，需要注意的是函数的形式参数 x 本质上即为 df_demo

In [ ]:
def condition(x):
   ondition_1_1 = x.School == 'Fudan University'
   condition_1_2 = x.Grade == 'Senior'
   condition_1_3 = x.Weight > 70
   condition_1 = condition_1_1 & condition_1_2 & condition_1_3
   condition_2_1 = x.School == 'Peking University'
   condition_2_2 = x.Grade == 'Senior'
   condition_2_3 = x.Weight > 80
   condition_2 = condition_2_1 & (~condition_2_2) & condition_2_3
   result = condition_1 | condition_2
   return result
df_demo.loc[condition]

In [33]:
# 还支持使用 lambda 表达式，其返回值也同样必须是先前提到的四种形式之一
df_demo.loc[lambda x:'Quan Zhao', lambda x:'Gender']

'Female'

函数无法返回如 start: end: step 的切片形式，故返回切片时要用 slice 对象进行包装

In [34]:
 df_demo.loc[lambda x: slice('Gaojuan You', 'Gaoqiang Qian')]

,School,Grade,Gender,Weight,Transfer
Name,,,,,
Gaojuan You,Fudan University,Sophomore,Male,74.0,N
Xiaoli Qian,Tsinghua University,Freshman,Female,51.0,N
Qiang Chu,Shanghai Jiao Tong University,Freshman,Female,52.0,N
Gaoqiang Qian,Tsinghua University,Junior,Female,50.0,N


！对于 Series 也可以使用 loc 索引，其遵循的原则与 DataFrame 中用于行筛选的 loc[*] 完全一致

注意！！不要使用链式赋值

在对表或者序列赋值时，应当在使用一层索引器后直接进行赋值操作

### 4.iloc索引器

iloc 的使用与 loc 完全类似，只不过是针对位置进行筛选

In [35]:
df_demo.iloc[1, 1] # 第二行第二列
df_demo.iloc[[0, 1], [0, 1]] # 前两行前两列
df_demo.iloc[1: 4, 2:4] # 切片不包含结束端点
df_demo.iloc[lambda x: slice(1, 4)] # 传入切片为返回值的函数

,School,Grade,Gender,Weight,Transfer
Name,,,,,
Changqiang You,Peking University,Freshman,Male,70.0,N
Mei Sun,Shanghai Jiao Tong University,Senior,Male,89.0,N
Xiaojuan Sun,Fudan University,Sophomore,Female,41.0,N


用布尔筛选的时候还是应当优先考虑 loc 的方式

因为：使用布尔列表的时候不能传入 Series ，必须传入序列的 values ，否则会报错

对 Series 而言同样也可以通过 iloc 返回相应位置的值或子序列

In [36]:
df_demo.School.iloc[1]
df_demo.School.iloc[1:5:2]

Name
Changqiang You    Peking University
Xiaojuan Sun       Fudan University
Name: School, dtype: object

### 5.query 方法

在 pandas 中，支持把字符串形式的查询表达式传入 query 方法来查询数据，其表达式的执行结果必须返回布尔列表

适合复杂检索，可在不降低可读性的前提下减少代码长度

In [37]:
# 将 loc 中的复合条件查询例子可以改写如下：
df.query('((School == "Fudan University")&'' (Grade == "Senior")&'' (Weight > 70))|''((School == "Peking University")&'' (Grade != "Senior")&'' (Weight > 80))')

,School,Grade,Name,Gender,Weight,Transfer
38,Peking University,Freshman,Qiang Han,Male,87.0,N
66,Fudan University,Senior,Chengpeng Zhou,Male,81.0,N
99,Peking University,Freshman,Changpeng Zhao,Male,83.0,N
131,Fudan University,Senior,Chengpeng Qian,Male,73.0,Y


在 query 表达式中，所有属于该 Series 的方法都可以被调用，和正常的函数调用并没有区别

！！对于含有空格的列名，需要使用`col name`的方式进行引用。

在 query 中还注册了若干英语的字面用法，帮助提高可读性，例如： or, and, or, is in, not in

 == 和 != 分别表示元素出现在列表和没有出现在列表，等价于 is in 和 not in

对于 query 中的字符串，如果要引用外部变量，只需在变量名前加 @ 符号

In [ ]:
low, high =70, 80
df.query('Weight.between(@low, @high)').head()

### 6.随机抽样

sample 函数中的主要参数为 n, axis, frac, replace, weights ，前三个分别是指抽样数量、抽样的方向（0为行、1为列）和抽样比例（0.3则为从总体中抽出30%的样本）

replace 和 weights 分别是指是否放回和每个样本的抽样相对概率，当 replace = True 则表示有放回抽样

In [ ]:
#对下面构造的 df_sample 以 value 值的相对大小为抽样概率进行有放回抽样，抽样数量为3
df_sample = pd.DataFrame({'id': list('abcde'),'value': [1, 2, 3, 4, 90]})
df_sample.sample(3, replace = True, weights = df_sample.value)

## 二、多级索引

### 1.多级索引及其表的结构

In [42]:
#构造一张新表
np.random.seed(0)
multi_index = pd.MultiIndex.from_product([list('ABCD'),df.Gender.unique()], names=('School', 'Gender'))
multi_column = pd.MultiIndex.from_product([['Height', 'Weight'],df.Grade.unique()], names=('Indicator', 'Grade'))
df_multi = pd.DataFrame(np.c_[(np.random.randn(8,4)*5 + 163).tolist(),(np.random.randn(8,4)*5 + 65).tolist()],index = multi_index,columns = multi_column).round(1)
df_multi

Indicator       Height                           Weight                        
Grade         Freshman Senior Sophomore Junior Freshman Senior Sophomore Junior
School Gender                                                                  
A      Female    171.8  165.0     167.9  174.2     60.6   55.1      63.3   65.8
       Male      172.3  158.1     167.8  162.2     71.2   71.0      63.1   63.5
B      Female    162.5  165.1     163.7  170.3     59.8   57.9      56.5   74.8
       Male      166.8  163.6     165.2  164.7     62.5   62.8      58.7   68.9
C      Female    170.5  162.0     164.6  158.7     56.9   63.9      60.5   66.9
       Male      150.2  166.3     167.3  159.3     62.4   59.1      64.9   67.1
D      Female    174.3  155.7     163.2  162.1     65.3   66.5      61.8   63.2
       Male      170.7  170.3     163.8  164.9     61.6   63.2      60.9   56.4

 DataFrame的行索引和列索引都是 MultiIndex 类型，只是索引中的一个元素是元组而不是单层索引中的标量

 这里需要注意，外层连续出现相同的值时，第一次之后出现的会被隐藏显示，使结果的可读性增强

索引的名字和值属性分别可以通过 names 和 values 获得

In [44]:
df_multi.index.names

FrozenList(['School', 'Gender'])

In [47]:
df_multi.index.values

array([('A', 'Female'), ('A', 'Male'), ('B', 'Female'), ('B', 'Male'),
       ('C', 'Female'), ('C', 'Male'), ('D', 'Female'), ('D', 'Male')],
      dtype=object)

In [45]:
df_multi.columns.names

FrozenList(['Indicator', 'Grade'])

In [46]:
df_multi.columns.values

array([('Height', 'Freshman'), ('Height', 'Senior'),
       ('Height', 'Sophomore'), ('Height', 'Junior'),
       ('Weight', 'Freshman'), ('Weight', 'Senior'),
       ('Weight', 'Sophomore'), ('Weight', 'Junior')], dtype=object)

要得到某一层的索引，则需要通过 get_level_values 获得

In [48]:
df_multi.index.get_level_values(0)

Index(['A', 'A', 'B', 'B', 'C', 'C', 'D', 'D'], dtype='object', name='School')

Index(['A', 'A', 'B', 'B', 'C', 'C', 'D', 'D'], dtype='object', name='School')

### 2.多级索引中的loc索引器

多级索引中的单个元素以元组为单位，因此之前在第一节介绍的 loc 和 iloc 方法完全可以照搬，只需把标量的位置替换成对应的元组

In [50]:
df_multi = df.set_index(['School', 'Grade'])
df_multi.head()

Name  ... Transfer
School                        Grade                      ...         
Shanghai Jiao Tong University Freshman     Gaopeng Yang  ...        N
Peking University             Freshman   Changqiang You  ...        N
Shanghai Jiao Tong University Senior            Mei Sun  ...        N
Fudan University              Sophomore    Xiaojuan Sun  ...        N
                              Sophomore     Gaojuan You  ...        N

[5 rows x 4 columns]

当传入元组列表或单个元组或返回前二者的函数时，需要先进行索引排序以避免性能警告

In [52]:
import warnings
with warnings.catch_warnings():
   warnings.filterwarnings('error')
   try:
     df_multi.loc[('Fudan University', 'Junior')].head()
   except Warning as w:
     Warning_Msg = w
Warning_Msg     

pandas.errors.PerformanceWarning('indexing past lexsort depth may impact performance.')

In [53]:
df_sorted = df_multi.sort_index()
df_sorted.loc[('Fudan University', 'Junior')].head()
df_sorted.loc[[('Fudan University', 'Senior'),('Shanghai Jiao Tong University', 'Freshman')]].head()

Name  Gender  Weight Transfer
School           Grade                                           
Fudan University Senior  Chengpeng Zheng  Female    38.0        N
                 Senior        Feng Zhou  Female    47.0        N
                 Senior        Gaomei Lv  Female    34.0        N
                 Senior        Chunli Lv  Female    56.0        N
                 Senior   Chengpeng Zhou    Male    81.0        N

注意！！当使用切片时需要注意，在单级别索引中只要切片端点元素是唯一的，那么就可以进行切片，但在多级索引中，无论元组在索引中是否重复出现，都必须经过排序才能使用切片，否则报错

注意！！在多级索引中的元组有一种特殊的用法，可以对多层的元素进行交叉组合后索引，但同时需要指定 loc 的列，全选则用 : 表示。其中，每一层需要选中的元素用列表存放，传入 loc 的形式为 [(level_0_list, level_1_list), cols] 

In [54]:
# 所有北大和复旦的大二大三学生
res = df_multi.loc[(['Peking University', 'Fudan University'],['Sophomore', 'Junior']), :]
res.head()

Name  Gender  Weight Transfer
School            Grade                                           
Peking University Sophomore   Changmei Xu  Female    43.0        N
                  Sophomore  Xiaopeng Qin    Male     NaN        N
                  Sophomore        Mei Xu  Female    39.0        N
                  Sophomore   Xiaoli Zhou  Female    55.0        N
                  Sophomore      Peng Han  Female    34.0      NaN

In [55]:
# 选出北大的大三学生和复旦的大二学生
res = df_multi.loc[[('Peking University', 'Junior'),('Fudan University', 'Sophomore')]]
res.head()

Name  Gender  Weight Transfer
School            Grade                                         
Peking University Junior        Juan Xu  Female     NaN        N
                  Junior  Changjuan You  Female    47.0        N
                  Junior       Gaoli Xu  Female    48.0        N
                  Junior   Gaoquan Zhou    Male    70.0        N
                  Junior      Qiang You  Female    56.0        N

### 3.IndexSlice对象

为了解决每层切片、切片和布尔列表混合使用的问题

Slice 对象一共有两种形式，第一种为 loc[idx[*,*]] 型，第二种为 loc[idx[*,*],idx[*,*]] 型

In [56]:
# 构造一个 索引不重复的 DataFrame
np.random.seed(0)
L1,L2 = ['A','B','C'],['a','b','c']
mul_index1 = pd.MultiIndex.from_product([L1,L2],names=('Upper', 'Lower'))
L3,L4 = ['D','E','F'],['d','e','f']
mul_index2 = pd.MultiIndex.from_product([L3,L4],names=('Big', 'Small'))
df_ex = pd.DataFrame(np.random.randint(-9,10,(9,9)),index=mul_index1,columns=mul_index2)
df_ex

Big          D        E        F      
Small        d  e  f  d  e  f  d  e  f
Upper Lower                           
A     a      3  6 -9 -6 -6 -2  0  9 -5
      b     -3  3 -8 -3 -2  5  8 -4  4
      c     -1  0  7 -4  6  6 -9  9 -6
B     a      8  5 -2 -9 -8  0 -9  1 -6
      b      2  9 -7 -9 -9 -5 -4 -3 -1
      c      8  6 -5  0  1 -8 -8 -2  0
C     a     -6 -3  2  5  9 -9  5 -6  3
      b      1  2 -5 -3 -5  6 -6  3 -5
      c     -1  5  6 -6  6  4  7  8 -4

In [57]:
idx = pd.IndexSlice #定义

（1） loc[idx[*,*]] 型

不能进行多层分别切片，前一个 * 表示行的选择，后一个 * 表示列的选择，与 loc 类似

In [58]:
df_ex.loc[idx['C':, ('D', 'f'):]]

Big          D  E        F      
Small        f  d  e  f  d  e  f
Upper Lower                     
C     a      2  5  9 -9  5 -6  3
      b     -5 -3 -5  6 -6  3 -5
      c      6 -6  6  4  7  8 -4

In [ ]:
df_ex.loc[idx[:'A', lambda x:x.sum()>0]] # 列和大于0，布尔序列的索引

（2） loc[idx[*,*],idx[*,*]] 型

能够分层进行切片，前一个 idx 指代的是行索引，后一个是列索引

In [59]:
 df_ex.loc[idx[:'A', 'b':], idx['E':, 'e':]]

Big          E     F   
Small        e  f  e  f
Upper Lower            
A     b     -2  5 -4  4
      c      6  6  9 -6

### 4.多级索引的构造

常用的有 from_tuples, from_arrays, from_product 三种方法，它们都是 pd.MultiIndex 对象下的函数

from_tuples 指根据传入由元组组成的列表进行构造

In [60]:
my_tuple = [('a','cat'),('a','dog'),('b','cat'),('b','dog')]
pd.MultiIndex.from_tuples(my_tuple, names=['First','Second'])

MultiIndex([('a', 'cat'),
            ('a', 'dog'),
            ('b', 'cat'),
            ('b', 'dog')],
           names=['First', 'Second'])

from_arrays 指根据传入列表中，对应层的列表进行构造

In [61]:
my_array = [list('aabb'), ['cat', 'dog']*2]
pd.MultiIndex.from_arrays(my_array, names=['First','Second'])

MultiIndex([('a', 'cat'),
            ('a', 'dog'),
            ('b', 'cat'),
            ('b', 'dog')],
           names=['First', 'Second'])

from_product 指根据给定多个列表的笛卡尔积进行构造

In [62]:
my_list1 = ['a','b']
my_list2 = ['cat','dog']
pd.MultiIndex.from_product([my_list1,my_list2],names=['First','Second'])

MultiIndex([('a', 'cat'),
            ('a', 'dog'),
            ('b', 'cat'),
            ('b', 'dog')],
           names=['First', 'Second'])

## 三、索引的常用方法

### 1.索引层的交换和删除

In [63]:
# 构造一个三级索引的例子
np.random.seed(0)
L1,L2,L3 = ['A','B'],['a','b'],['alpha','beta']
mul_index1 = pd.MultiIndex.from_product([L1,L2,L3],names=('Upper', 'Lower','Extra'))
L4,L5,L6 = ['C','D'],['c','d'],['cat','dog']
mul_index2 = pd.MultiIndex.from_product([L4,L5,L6],names=('Big', 'Small', 'Other'))
df_ex = pd.DataFrame(np.random.randint(-9,10,(8,8)),index=mul_index1,columns=mul_index2)
df_ex

Big                 C               D            
Small               c       d       c       d    
Other             cat dog cat dog cat dog cat dog
Upper Lower Extra                                
A     a     alpha   3   6  -9  -6  -6  -2   0   9
            beta   -5  -3   3  -8  -3  -2   5   8
      b     alpha  -4   4  -1   0   7  -4   6   6
            beta   -9   9  -6   8   5  -2  -9  -8
B     a     alpha   0  -9   1  -6   2   9  -7  -9
            beta   -9  -5  -4  -3  -1   8   6  -5
      b     alpha   0   1  -8  -8  -2   0  -6  -3
            beta    2   5   9  -9   5  -6   3   1

索引层的交换由 swaplevel 和 reorder_levels 完成，前者只能交换两个层，而后者可以交换任意层，两者都可以指定交换的是轴是哪一个，即行索引或列索引

In [ ]:
 df_ex.swaplevel(0,2,axis=1).head() # 列索引的第一层和第三层交换
 df_ex.reorder_levels([2,0,1],axis=0).head() # 列表数字指代原来索引中的层

想要删除某一层的索引，可以使用 droplevel 方法

In [64]:
df_ex.droplevel(1,axis=1)

Big                 C               D            
Other             cat dog cat dog cat dog cat dog
Upper Lower Extra                                
A     a     alpha   3   6  -9  -6  -6  -2   0   9
            beta   -5  -3   3  -8  -3  -2   5   8
      b     alpha  -4   4  -1   0   7  -4   6   6
            beta   -9   9  -6   8   5  -2  -9  -8
B     a     alpha   0  -9   1  -6   2   9  -7  -9
            beta   -9  -5  -4  -3  -1   8   6  -5
      b     alpha   0   1  -8  -8  -2   0  -6  -3
            beta    2   5   9  -9   5  -6   3   1

In [ ]:
df_ex.droplevel([0,1],axis=0)

### 2.索引属性的修改

rename_axis 可以对索引层的名字进行修改，常用的修改方式是传入字典的映射

In [65]:
df_ex.rename_axis(index={'Upper':'Changed_row'},columns={'Other':'Changed_Col'}).head()

Big                       C               D            
Small                     c       d       c       d    
Changed_Col             cat dog cat dog cat dog cat dog
Changed_row Lower Extra                                
A           a     alpha   3   6  -9  -6  -6  -2   0   9
                  beta   -5  -3   3  -8  -3  -2   5   8
            b     alpha  -4   4  -1   0   7  -4   6   6
                  beta   -9   9  -6   8   5  -2  -9  -8
B           a     alpha   0  -9   1  -6   2   9  -7  -9

通过 rename 可以对索引的值进行修改，如果是多级索引需要指定修改的层号 level

In [66]:
df_ex.rename(columns={'cat':'not_cat'},level=2).head()

Big                     C                       D                
Small                   c           d           c           d    
Other             not_cat dog not_cat dog not_cat dog not_cat dog
Upper Lower Extra                                                
A     a     alpha       3   6      -9  -6      -6  -2       0   9
            beta       -5  -3       3  -8      -3  -2       5   8
      b     alpha      -4   4      -1   0       7  -4       6   6
            beta       -9   9      -6   8       5  -2      -9  -8
B     a     alpha       0  -9       1  -6       2   9      -7  -9

传入参数也可以是函数，其输入值就是索引元素

In [67]:
df_ex.rename(index=lambda x:str.upper(x),level=2).head()

Big                 C               D            
Small               c       d       c       d    
Other             cat dog cat dog cat dog cat dog
Upper Lower Extra                                
A     a     ALPHA   3   6  -9  -6  -6  -2   0   9
            BETA   -5  -3   3  -8  -3  -2   5   8
      b     ALPHA  -4   4  -1   0   7  -4   6   6
            BETA   -9   9  -6   8   5  -2  -9  -8
B     a     ALPHA   0  -9   1  -6   2   9  -7  -9

对于整个索引的元素替换，可以利用迭代器实现

In [69]:
new_values = iter(list('abcdefgh'))
df_ex.rename(index=lambda x:next(new_values),level=2)

Big                 C               D            
Small               c       d       c       d    
Other             cat dog cat dog cat dog cat dog
Upper Lower Extra                                
A     a     a       3   6  -9  -6  -6  -2   0   9
            b      -5  -3   3  -8  -3  -2   5   8
      b     c      -4   4  -1   0   7  -4   6   6
            d      -9   9  -6   8   5  -2  -9  -8
B     a     e       0  -9   1  -6   2   9  -7  -9
            f      -9  -5  -4  -3  -1   8   6  -5
      b     g       0   1  -8  -8  -2   0  -6  -3
            h       2   5   9  -9   5  -6   3   1

 map：定义在 Index 上的方法，它传入的不是层的标量值，而是直接传入索引的元组，为跨层的修改提供了遍历

 

In [70]:
df_temp = df_ex.copy()
new_idx = df_temp.index.map(lambda x: (x[0],x[1],str.upper(x[2])))
df_temp.index = new_idx
df_temp.head()

Big                 C               D            
Small               c       d       c       d    
Other             cat dog cat dog cat dog cat dog
Upper Lower Extra                                
A     a     ALPHA   3   6  -9  -6  -6  -2   0   9
            BETA   -5  -3   3  -8  -3  -2   5   8
      b     ALPHA  -4   4  -1   0   7  -4   6   6
            BETA   -9   9  -6   8   5  -2  -9  -8
B     a     ALPHA   0  -9   1  -6   2   9  -7  -9

map 的另一个使用方法是对多级索引的压缩

In [71]:
df_temp = df_ex.copy()
new_idx = df_temp.index.map(lambda x: (x[0]+'-'+x[1]+'-'+x[2]))
df_temp.index = new_idx
df_temp.head() # 单层索引

Big         C               D            
Small       c       d       c       d    
Other     cat dog cat dog cat dog cat dog
A-a-alpha   3   6  -9  -6  -6  -2   0   9
A-a-beta   -5  -3   3  -8  -3  -2   5   8
A-b-alpha  -4   4  -1   0   7  -4   6   6
A-b-beta   -9   9  -6   8   5  -2  -9  -8
B-a-alpha   0  -9   1  -6   2   9  -7  -9

反向展开

In [72]:
new_idx = df_temp.index.map(lambda x:tuple(x.split('-')))
df_temp.index = new_idx
df_temp.head() # 三层索引

Big         C               D            
Small       c       d       c       d    
Other     cat dog cat dog cat dog cat dog
A a alpha   3   6  -9  -6  -6  -2   0   9
    beta   -5  -3   3  -8  -3  -2   5   8
  b alpha  -4   4  -1   0   7  -4   6   6
    beta   -9   9  -6   8   5  -2  -9  -8
B a alpha   0  -9   1  -6   2   9  -7  -9

### 3.索引的设置与重置

In [73]:
# 构造一个新表
df_new = pd.DataFrame({'A':list('aacd'),'B':list('PQRT'),'C':[1,2,3,4]})
df_new

索引的设置可以使用 set_index 完成，这里的主要参数是 append ，表示是否来保留原来的索引，直接把新设定的添加到原索引的内层

In [74]:
df_new.set_index('A')
df_new.set_index('A', append=True)

,,B,C
,A,,
0,a,P,1
1,a,Q,2
2,c,R,3
3,d,T,4


同时指定多个列作为索引

In [75]:
df_new.set_index(['A', 'B'])

C
A B   
a P  1
  Q  2
c R  3
d T  4

如果想要添加索引的列没有出现再其中，那么可以直接在参数中传入相应的 Series 

In [76]:
my_index = pd.Series(list('WXYZ'), name='D')
df_new = df_new.set_index(['A', my_index])
df_new

B  C
A D      
a W  P  1
  X  Q  2
c Y  R  3
d Z  T  4

reset_index 是 set_index 的逆函数，其主要参数是 drop ，表示是否要把去掉的索引层丢弃

In [ ]:
df_new.reset_index(['D'])
df_new.reset_index(['D'], drop=True)

如果重置了所有的索引，那么 pandas 会直接重新生成一个默认索引

In [77]:
df_new.reset_index()

,A,D,B,C
0,a,W,P,1
1,a,X,Q,2
2,c,Y,R,3
3,d,Z,T,4


### 4.索引的变形

 reindex 

在某些场合下，需要对索引做一些扩充或者剔除，更具体地要求是给定一个新的索引，把原表中相应的索引对应元素填充到新索引构成的表中

这种需求常出现在时间序列索引的时间点填充以及 ID 编号的扩充

与 reindex 功能类似的函数是 reindex_like ，其功能是仿照传入的表的索引来进行被调用表索引的变形

## 四、索引运算

### 1.集合的运算法则

并、交、差、补

id1 & id2

id1 | id2

(id1 ^ id2) & id1

id1 ^ id2 # ^符号即对称差

### 2.一般的索引运算

由于集合的元素是互异的，但是索引中可能有相同的元素，先用 unique 去重后再进行运算

若两张表需要做集合运算的列并没有被设置索引，一种办法是先转成索引，运算后再恢复，另一种方法是利用 isin 函数

## 五、练习


### EX1：公司员工数据集

In [78]:
df = pd.read_csv('data/company.csv')
df.head(3)

,EmployeeID,birthdate_key,age,city_name,department,job_title,gender
0,1318,1/3/1954,61,Vancouver,Executive,CEO,M
1,1319,1/3/1957,58,Vancouver,Executive,VP Stores,F
2,1320,1/2/1955,60,Vancouver,Executive,Legal Counsel,F


In [79]:
#Q1
dpt = ['Dairy', 'Bakery']
df.query("(age <= 40)&(department == @dpt)&(gender=='M')").head(3)

,EmployeeID,birthdate_key,age,city_name,department,job_title,gender
3611,5791,1/14/1975,40,Kelowna,Dairy,Dairy Person,M
3613,5793,1/22/1975,40,Richmond,Bakery,Baker,M
3615,5795,1/30/1975,40,Nanaimo,Dairy,Dairy Person,M


In [80]:
#Q1
df.loc[(df.age<=40)&df.department.isin(dpt)&(df.gender=='M')].head(3)

,EmployeeID,birthdate_key,age,city_name,department,job_title,gender
3611,5791,1/14/1975,40,Kelowna,Dairy,Dairy Person,M
3613,5793,1/22/1975,40,Richmond,Bakery,Baker,M
3615,5795,1/30/1975,40,Nanaimo,Dairy,Dairy Person,M


In [81]:
# Q2
df.iloc[(df.EmployeeID%2==1).values,[0,2,-2]].head()

,EmployeeID,age,job_title
1,1319,58,VP Stores
3,1321,56,VP Human Resources
5,1323,53,"Exec Assistant, VP Stores"
6,1325,51,"Exec Assistant, Legal Counsel"
8,1329,48,Store Manager


In [ ]:
#Q3


### EX2：巧克力数据集

In [82]:
df = pd.read_csv('data/chocolate.csv')
df.head(3)

,Company,Review\nDate,Cocoa\nPercent,Company\nLocation,Rating
0,A. Morin,2016,63%,France,3.75
1,A. Morin,2015,70%,France,2.75
2,A. Morin,2015,70%,France,3.00


In [83]:
#Q1
df.columns = [' '.join(i.split('\n')) for i in df.columns]
df.head(3)

,Company,Review Date,Cocoa Percent,Company Location,Rating
0,A. Morin,2016,63%,France,3.75
1,A. Morin,2015,70%,France,2.75
2,A. Morin,2015,70%,France,3.00
